### Playing Audio

> Initialize **media player**

> Set up **audio manager** to control volume levels

> Set up **Seek bar** (more details at end of section) in the layout and synchronize with volume levels and audio progress

> Place a sample audio file in a folder named **raw** under **res**

> **Getting resource ID**

>> We can then access the resource id with **R.raw.filename** (which is an integer)

>> But most often, we want to get the resource id from **file name**, in which case we use
```java
 int resourceId = getResources().getIdentifier(filename,"raw",getPackageName());
```

```java
package com.aravindsankaran.audio;

import android.content.Context;
import android.media.AudioManager;
import android.media.MediaPlayer;
import android.support.v7.app.AppCompatActivity;
import android.os.Bundle;
import android.util.Log;
import android.view.View;
import android.widget.SeekBar;

import java.util.Timer;
import java.util.TimerTask;


public class MainActivity extends AppCompatActivity {

    MediaPlayer mplayer;
    AudioManager audioManager;


    //for play button in the layout
    public void onPlay(View view){
        mplayer.start();
    }

    //for pause button in the layout
    public void onPause(View view){
        //When we pause when the media is not playing, we get start-exception.
        // Ie, next time we call start, it may not work
        if(mplayer.isPlaying())
            mplayer.pause();
    }

    //for restart button in the layout
    public void onRestart(View view){
        if(mplayer.isPlaying())
            mplayer.pause();
        mplayer.seekTo(0);

    }


    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_main);

        //when 'this' is not available, use MainActivity.this or getApplicationContext()                                                
        mplayer = MediaPlayer.create(this,R.raw.sample);

        audioManager = (AudioManager) getSystemService(Context.AUDIO_SERVICE);

        //Get the max volume set in your device for media (music)
        int maxVolume = audioManager.getStreamMaxVolume(AudioManager.STREAM_MUSIC);
        int currentVolume = audioManager.getStreamVolume(AudioManager.STREAM_MUSIC);


        //synchronize with volume with seek bar
        SeekBar volumeControl = (SeekBar) findViewById(R.id.seekBar6);

        //set the max volume as the max of seekbar, which is otherwise between 0 and 100
        volumeControl.setMax(maxVolume);
        //also set the current volume value
        volumeControl.setProgress(currentVolume);

        // new Object() {} : creates an anonymous class extending Object and creates and instance of that class
        volumeControl.setOnSeekBarChangeListener(new SeekBar.OnSeekBarChangeListener() {
            @Override
            public void onProgressChanged(SeekBar seekBar, int progress, boolean fromUser) {
                //progress changes from 0 to 100
                Log.i("volume lvl",Integer.toString(progress));

                //change volume
                audioManager.setStreamVolume(AudioManager.STREAM_MUSIC,progress,0);
            }

            @Override
            public void onStartTrackingTouch(SeekBar seekBar) {
                //abstract class and needs to be implemented
            }

            @Override
            public void onStopTrackingTouch(SeekBar seekBar) {
                //abstract class and needs to be implemented
            }
        });

        //synchronize progress
        final SeekBar progressControl = (SeekBar) findViewById(R.id.seekBar2);

        //set max duration on progress bar
        int duration = mplayer.getDuration();
        progressControl.setMax(duration);

        //sync progress bar with music play
        //this code carries out a specific task every 1s
        new Timer().scheduleAtFixedRate(new TimerTask() {
            @Override
            public void run() {
                progressControl.setProgress(mplayer.getCurrentPosition());
            }
        },0,1000);

        progressControl.setOnSeekBarChangeListener(new SeekBar.OnSeekBarChangeListener() {
            @Override
            public void onProgressChanged(SeekBar seekBar, int progress, boolean fromUser) {
                mplayer.seekTo(progress);
            }

            @Override
            public void onStartTrackingTouch(SeekBar seekBar) {

            }

            @Override
            public void onStopTrackingTouch(SeekBar seekBar) {

            }
        });
    }
}
```

### Playing video

> Set up **video view** in the layout

> Set the path to the video file under **raw** in the java file as
```java
videoView.setVideoPath("android.resource://" + getPackageName() + "/" + R.raw.sample1);
```

> Use media controller to control playbacks



```java
public class MainActivity extends AppCompatActivity {

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_main);

        VideoView videoView = (VideoView) findViewById(R.id.videoView);

        videoView.setVideoPath("android.resource://" + getPackageName() + "/" + R.raw.sample1);
        //videoView.setVideoPath("android.resource://" + getPackageName() + "/" + R.raw.screencast); //not playing

        MediaController mediaController = new MediaController(this);
        mediaController.setAnchorView(videoView);

        videoView.setMediaController(mediaController);

        //get access to media player from video view
//        videoView.setOnPreparedListener(new MediaPlayer.OnPreparedListener() {
//            @Override
//            public void onPrepared(MediaPlayer mp) {
                  //get access to media player
//                //mp.setVolume(30f,30f);
//            }
//        });

        videoView.start();

    }
}
```

### Binding media player to media controller

> Instead of setting your own play pause buttons in the layout while playing audio, you can bind media player and media controller (**implement MediaPlayerControl class** and implelent the abstract methods for play,pause,stop etc

> Refer [here](https://stackoverflow.com/questions/2961749/mediacontroller-with-mediaplayer)

### Note on Seek bar

> It is not possible to set minimum value in seek bar. It is also not possible to change the step size 1

> How ever, we can get around this by simple math

```java
int step = 1;
int max = 180;
int min = 60;

// Ex : 
// If you want values from 3 to 5 with a step of 0.1 (3, 3.1, 3.2, ..., 5)
// this means that you have 21 possible values in the seekbar.
// So the range of the seek bar will be [0 ; (5-3)/0.1 = 20].
seekbar.setMax( (max - min) / step );


seekbar.setOnSeekBarChangeListener(
    new OnSeekBarChangeListener()
    {
        @Override
        public void onStopTrackingTouch(SeekBar seekBar) {}

        @Override
        public void onStartTrackingTouch(SeekBar seekBar) {}

        @Override
        public void onProgressChanged(SeekBar seekBar, int progress, 
            boolean fromUser) 
        {
            // Ex :
            // And finally when you want to retrieve the value in the range you
            // wanted in the first place -> [3-5]
            //
            // if progress = 13 -> value = 3 + (13 * 0.1) = 4.3
            double value = min + (progress * step);

        }
    }
);
```

#### Disable seekbar based on some global boolean variable

```java
        boolean timerAvtive = false
    
       //disable seekbar dragging when timer is on
        timeBar.setOnTouchListener(new View.OnTouchListener() {
            @Override
            public boolean onTouch(View v, MotionEvent event) {
                //true : cannot drag. ie widget is always on touch by event
                //false: can drag
                return timerActive;
            }
        });
```